In [ ]:
import numpy as np

In [ ]:
# 생성자의 출력층 GenOutLayer 클래스 
class GenOutLayer(BaseLayer):
    def __init__(self, n_upper, n):
        self.w = np.random.randn(n_upper, n) / np.sqrt(n_upper)
        self.b = np.zeros(n)

    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = np.tanh(u)

    def backward(self, grad_y):
        delta = grad_y * (1 - self.y**2)

        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        self.grad_x = np.dot(delta, self.w.T)

In [ ]:
# 식별자의 출력층
class DiscOutLayer(BaseLayer):
    def __init__(self, n_upper, n):
        self.w = np.random.randn(n_upper, n)
        self.b = np.zeros(n)

    def forward(self, x):
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = 1/(1+np.exp(-u))

    def backward(self, t):
        delta = self.y-t

        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.dot(delta, axis=0)
        self.grad_x = np.dot(delta self.w.T)

In [ ]:
## 순전파와 역전파의 구현

#--각 신경망의 초기화--
gen_layers = [MiddleLayer(n_noise, 32),
              MiddleLayer(32,64),
              GenOutLayer(64, img_size*img_size)]

disc_layers = [MiddleLayer(img_size*img_size),
               MiddleLayer(64,32),
               GenOutLayer(32,1)]

#--순전파 함수--
def forward_propagation(x, layers):
    for layer in layers:
        layer.forward(x)
        x = layer.y
    return x

#--역전파 함수--
def backpropagation(t, layers):
    grad_y = t
    for layer in reversed(layers):
        layer.backward(grad_y)
        grad_y = layer.grad_X
    return grad_y

In [ ]:
# GAN의 훈련
def train_model(x, t, prop_layers, update_layers):
    y = forward_propagation(x, prop_layers)
    backpropagation(t, prop_layers)
    update_params(update_layers)

    return(get_error(y,t), get_accuracy(y,t))

In [ ]:
# GAN의 학습 코드
batch_half = batch_size // 2
error_record = np.zeros((n_learn, 2))
acc_record = np.zeros((n_learn, 2))

for i in range(n_learn):
    
    #노이즈에서 이미지를 생성하여 식별자를 훈련
    noise = np.random.normal(0,1,(batch_half, n_noise))
    imgs_fake = forward_propagation(noise, gen_layers)
    t = np.zeros((batch_half, 1))
    error, accuracy = train_model(imgs_fake, t, disc_layers, disc_layers)
    error_record[i][0] = error
    acc_record[i][0] = accuracy
    
    #실제이미지를 사용하여 식별자를 훈련
    rand_ids = np.random.randint(len(x_train), size=batch_half)
    imgs_real = x_train[rand_ids, :]
    t = np.ones((batch_half, 1))
    error, accuracy = train_model(imgs_real, t, disc_layers, disc_layers)
    error_record[i][1] = error
    acc_record[i][1] = accuracy

    #생성자와 식별자 모델을 결합해 생성자를 훈련
    noise = np.random.normal(0,1,(batch_size, n_noise))
    t = np.ones((batch_size, 1))
    train_model(noise, t, gen_layers+disc_layers, gen_layers)

In [ ]:
# 이미지 생성 코드

def generate_images(i):
    n_rows = 16
    n_cols = 16
    noise = np.random.normal(0,1,(n_rows*n_cols, n_noise))
    g_imgs = forward_propagation(noise, gen_layers)
    g_imgs = g_img/2 + 0.5

    img_size_spaced = img_size + 2

    matrix_image = np.zeros((img_size_spaced*n_rows, img_size_spaced*n_cols))

    for r in range(n_rows):
        for c in range(n_cols):
            g_img = g_imgs[r*n_cols + c].reshape(img_size, img_size)
            top = r*img_size_spaced
            left = c*img_size_spaced
            matrix_image[top : top+img_size, left : left+img_size] = g_img


    plt.figure(figsize=(8,8))
    plt.imshow(matrix_image.tolist(), cmap='Greys_r')
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)
    plt.show()